In [4]:
import cdsapi
import xarray as xr
import numpy as np
from tempfile import NamedTemporaryFile

# Initialize the CDS API client
c = cdsapi.Client()

# Define parameters for the data request
location = {'latitude': 31.70, 'longitude': 6.015}  # Example: Algeria location
date = '2021-01-01'

# Retrieve ERA5 data and store it in a temporary file
with NamedTemporaryFile(suffix='.nc') as tmp_file:
    result = c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind'],
            'year': date.split('-')[0],
            'month': date.split('-')[1],
            'day': date.split('-')[2],
            'time': ['10:00'],  # Specify time of interest
            'format': 'netcdf',  # NetCDF format
            'area': [
                location['latitude'] + 0.25, location['longitude'] - 0.25,
                location['latitude'] - 0.25, location['longitude'] + 0.25,
            ],  # Small bounding box around the location
        }
    )
    # Download data to the temporary file
    result.download(tmp_file.name)
    
    # Load the dataset with xarray
    ds = xr.open_dataset(tmp_file.name)

# Extract u and v components
u10 = ds['u10'].sel(latitude=location['latitude'], longitude=location['longitude'], method='nearest')
v10 = ds['v10'].sel(latitude=location['latitude'], longitude=location['longitude'], method='nearest')

# Calculate wind speed
wind_speed = np.sqrt(u10**2 + v10**2)

# Handle single timestep case
if 'time' in u10.dims:
    # Multiple timesteps (not likely in this case since we specified 10:00 only)
    for time, speed in zip(u10.time.values, wind_speed.values):
        print(f"{time}: Wind Speed = {speed:.2f} m/s")
else:
    # Single timestep
    wind_speed_value = wind_speed.values.item()  # Convert array to scalar
    print(f"Wind Speed at 10:00 on {date}: {wind_speed_value:.2f} m/s")



2025-01-03 18:40:55,827 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2025-01-03 18:40:55,827 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-01-03 18:40:55,828 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2025-01-03 18:40:55,829 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using t

9fd2ab615ce3a9e26e934e38a48f826d.nc:   0%|          | 0.00/33.3k [00:00<?, ?B/s]

Wind Speed at 10:00 on 2021-01-01: 2.10 m/s
